# Code to train a network to detect false images

### load dependencies

In [2]:
import torch
import torchvision.models as models
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn

## Load efficientnet

In [4]:
# EfficientNet-B0 laden (du kannst B1-B7 verwenden, je nach Größe)
model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.IMAGENET1K_V1)

# Anzahl der Klassen in ImageNet
num_classes = 200

# Klassifikationskopf anpassen (falls nötig)
model.classifier[1] = torch.nn.Linear(model.classifier[1].in_features, num_classes)

# Modell auf GPU verschieben (falls verfügbar)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [7]:


# ImageNet-Transformationen (normalisieren wie beim vortrainierten Modell)
imagenet_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# ImageNet-Trainingsdaten laden
train_dataset = datasets.ImageNet(root="path/to/imagenet", split="train", transform=imagenet_transforms)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)


RuntimeError: The archive ILSVRC2012_devkit_t12.tar.gz is not present in the root directory or is corrupted. You need to download it externally and place it in path/to/imagenet.

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
num_epochs = 5

for epoch in range(num_epochs):
    model.train()  # Training aktivieren
    running_loss = 0.0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}")

print("Training abgeschlossen!")
